# Launching FastAI p2 instance using boto3

I wanted an idempotent launcher (doesn't create redundant resources when I run it again.)  Alternatively, I could have used CloudFormation.  See [this repo](https://github.com/scicolabs/jupyter-aws/blob/master/jupyter.template) for an example template that does similar things.

In [2]:
import os
import boto3
aws_session = boto3.session.Session()
region = aws_session.region_name

In [3]:
INSTANCE_TYPE = "p2.xlarge"
NAME="fast-ai"
CIDR="0.0.0.0/0"
CIDR_BLOCK="10.0.0.0/28"

```bash
export region=$(aws configure get region)
if [ $region = "us-west-2" ]; then
   export ami="ami-bc508adc" # Oregon
elif [ $region = "eu-west-1" ]; then
   export ami="ami-b43d1ec7" # Ireland
elif [ $region = "us-east-1" ]; then
  export ami="ami-31ecfb26" # Virginia
else
  echo "Only us-west-2 (Oregon), eu-west-1 (Ireland), and us-east-1 (Virginia) are currently supported"
  exit 1
fi

export instanceType="p2.xlarge"
```

See [this link](https://boto3.readthedocs.io/en/latest/guide/quickstart.html#configuration) for information on how to configure `boto3`

In [4]:
AMI_OF = {
    "us-west-2": "ami-bc508adc", #My pre-built AMI: ami-02cd7b7a
    "us-east-1": "ami-31ecfb26", 
    "eu-west-1": "ami-b43d1ec7"
}

if region not in AMI_OF:
    raise ValueError("No AMI available for AWS region {0}".format(region))
else:
    ami = AMI_OF[region]

# Create and configure VPC

```bash
export vpcId=$(aws ec2 create-vpc --cidr-block 10.0.0.0/28 --query 'Vpc.VpcId' --output text)
aws ec2 create-tags --resources $vpcId --tags --tags Key=Name,Value=$name
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-support "{\"Value\":true}"
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-hostnames "{\"Value\":true}"
```

In [5]:
ec2 = boto3.resource('ec2')
filters = [{'Name': 'tag:Name', 'Values': [NAME]}]
vpcs = list(ec2.vpcs.filter(Filters=filters))

In [6]:
if vpcs:
    print("VPC '{0}' exists, skipping creation".format(NAME))
    vpc = vpcs[0]
else:
    print("Creating VPC '{0}'".format(NAME))
    vpc = ec2.create_vpc(CidrBlock=CIDR_BLOCK)
    vpc.create_tags(Tags=[{"Key": "Name", "Value": NAME}])
    response = vpc.modify_attribute(
      EnableDnsSupport={
          'Value': True
      })
    response = response = vpc.modify_attribute(
      EnableDnsHostnames={
          'Value': True
      })
vpc

Creating VPC 'fast-ai'


ec2.Vpc(id='vpc-d522c1ac')

# Create and configure gateway

```bash
export internetGatewayId=$(aws ec2 create-internet-gateway --query 'InternetGateway.InternetGatewayId' --output text)
aws ec2 create-tags --resources $internetGatewayId --tags --tags Key=Name,Value=$name-gateway
aws ec2 attach-internet-gateway --internet-gateway-id $internetGatewayId --vpc-id $vpcId
```

In [10]:
filters = [{'Name': 'tag:Name', "Values": [NAME + "-gateway"]},
           {'Name': 'attachment.vpc-id', "Values": [vpc.id]}]

internet_gateways = list(ec2.internet_gateways.filter(Filters=filters))

if internet_gateways:
    print("Internet Gateway already exists for VPC Id {0}".format(vpc.id))
    gateway = internet_gateways[0]
else:
    print("Creating Internet Gateway {0} and attaching to {1}".format(NAME+"-gateway", vpc.id))
    gateway = ec2.create_internet_gateway()
    gateway.create_tags(Tags=[{"Key": "Name", "Value": NAME + "-gateway"}])
    vpc.attach_internet_gateway(InternetGatewayId=gateway.id)

gateway

Internet Gateway already exists for VPC Id vpc-d522c1ac


ec2.InternetGateway(id='igw-b5ab46d3')

# Create and configure subnet

```bash
export subnetId=$(aws ec2 create-subnet --vpc-id $vpcId --cidr-block 10.0.0.0/28 --query 'Subnet.SubnetId' --output text)
aws ec2 create-tags --resources $subnetId --tags --tags Key=Name,Value=$name-subnet
```

In [13]:
filters = [{'Name': 'tag:Name', "Values": [NAME + "-subnet"]}]
subnets = list(ec2.subnets.filter(Filters=filters))
 
def validate(subnet, vpc):
    return (subnet.cidr_block == CIDR_BLOCK) and (subnet.vpc_id == vpc.id)

if subnets and validate(subnets[0], vpc):
    print("Subnet already exists for VPC Id {0}".format(vpc.id))
    subnet = subnets[0]
else:
    print("Creating Subnet {0} and attaching to {1}".format(NAME+"-subnet", vpc.id))
    subnet = ec2.create_subnet(CidrBlock='10.0.0.0/28', VpcId=vpc.id)
    subnet.create_tags(Tags=[{"Key": "Name", "Value": NAME + "-subnet"}])

subnet

Subnet already exists for VPC Id vpc-d522c1ac


ec2.Subnet(id='subnet-4e82cd06')

# Configure routing table

```bash
export routeTableId=$(aws ec2 create-route-table --vpc-id $vpcId --query 'RouteTable.RouteTableId' --output text)
aws ec2 create-tags --resources $routeTableId --tags --tags Key=Name,Value=$name-route-table
export routeTableAssoc=$(aws ec2 associate-route-table --route-table-id $routeTableId --subnet-id $subnetId --output text)
aws ec2 create-route --route-table-id $routeTableId --destination-cidr-block 0.0.0.0/0 --gateway-id $internetGatewayId
```

In [28]:
DESTINATION_CIDR_BLOCK="0.0.0.0/0"
filters = [{'Name': 'tag:Name', "Values": [NAME + "-route-table"]},
           {'Name': 'association.subnet-id', "Values": [subnet.id]},
           {'Name': 'vpc-id', "Values": [vpc.id]},
           {'Name': 'route.destination-cidr-block', "Values": [DESTINATION_CIDR_BLOCK]},
           {'Name': 'route.gateway-id', "Values": [gateway.internet_gateway_id]}
          ]
route_tables = list(ec2.route_tables.filter(Filters=filters))

if route_tables:
    print("Route table already exists for VPC Id {0}".format(vpc.id))
    route_table = route_tables[0]
else:
    print("Creating Route Table {0} and associating to subnet {1} and gateway {2}".format(NAME+"-route-table", subnet.id, gateway.internet_gateway_id))
    
    route_table = ec2.create_route_table(VpcId=vpc.id)
    route_table.create_tags(Tags=[{"Key": "Name", "Value": NAME + "-route_table"}])
    route_table.associate_with_subnet(SubnetId=subnet.id)
    route_table.create_route(DestinationCidrBlock=DESTINATION_CIDR_BLOCK, GatewayId=gateway.id)


Creating Route Table fast-ai-route-table and associating to subnet subnet-4e82cd06 and gateway igw-b5ab46d3


# Configure security groups

```bash
export securityGroupId=$(aws ec2 create-security-group --group-name $name-security-group --description "SG for fast.ai machine" --vpc-id $vpcId --query 'GroupId' --output text)
# ssh
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 22 --cidr $cidr
# jupyter notebook
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 8888-8898 --cidr $cidr
```

In [39]:
filters = [
           {'Name': 'vpc-id', "Values": [vpc.id]},
           {'Name': 'ip-permission.cidr', "Values": [CIDR]},
           {'Name': 'ip-permission.to-port', "Values": ['22', '8898']}
          ]
security_groups = list(ec2.security_groups.filter(Filters=filters))

if security_groups:
    print("Security Group already exists for VPC Id {0}".format(vpc.id))
    security_group = security_groups[0]
else:
    print("Creating Security Group in VPC {0}".format(vpc.id))
    
    security_group = ec2.create_security_group(VpcId=vpc.id, 
                                               Description="SG for fast.ai machine",
                                               GroupName=NAME+"-security-group")
    security_group.authorize_ingress(CidrIp=CIDR, FromPort=22, ToPort=22, IpProtocol="tcp")
    security_group.authorize_ingress(CidrIp=CIDR, FromPort=8888, ToPort=8898, IpProtocol="tcp")

security_group

Security Group already exists for VPC Id vpc-d522c1ac


ec2.SecurityGroup(id='sg-c0ff4cbc')

# Set up keypairs

```bash
if [ ! -d ~/.ssh ]
then
	mkdir ~/.ssh
fi

if [ ! -f ~/.ssh/aws-key-$name.pem ]
then
	aws ec2 create-key-pair --key-name aws-key-$name --query 'KeyMaterial' --output text > ~/.ssh/aws-key-$name.pem
	chmod 400 ~/.ssh/aws-key-$name.pem
fi
```

In [50]:
import os
import stat

In [58]:
NAME = "foobarbaz"
if not os.path.exists(os.path.expanduser("~/.ssh")):
    os.mkdir(os.path.expanduser("~/.ssh"))

keyfile = os.path.expanduser("~/.ssh/aws-key-{0}.pem".format(NAME))
if not os.path.isfile(keyfile):
    key_pair = ec2.create_key_pair(KeyName="aws-key-" + NAME)
    with open(keyfile, mode='w') as f:
        f.write(key_pair.key_material)
    os.chmod(keyfile, 0o400)
                      

In [60]:
key_pair.delete()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Mon, 11 Dec 2017 00:13:41 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '3724dca4-475f-41af-b553-7724fe57185f',
  'RetryAttempts': 0}}

In [56]:
0o644

420

# Launch instance

```bash
export instanceId=$(aws ec2 run-instances --image-id $ami --count 1 --instance-type $instanceType --key-name aws-key-$name --security-group-ids $securityGroupId --subnet-id $subnetId --associate-public-ip-address --block-device-mapping "[ { \"DeviceName\": \"/dev/sda1\", \"Ebs\": { \"VolumeSize\": 128, \"VolumeType\": \"gp2\" } } ]" --query 'Instances[0].InstanceId' --output text)
aws ec2 create-tags --resources $instanceId --tags --tags Key=Name,Value=$name-gpu-machine
export allocAddr=$(aws ec2 allocate-address --domain vpc --query 'AllocationId' --output text)

echo Waiting for instance start...
aws ec2 wait instance-running --instance-ids $instanceId
sleep 10 # wait for ssh service to start running too
export assocId=$(aws ec2 associate-address --instance-id $instanceId --allocation-id $allocAddr --query 'AssociationId' --output text)
export instanceUrl=$(aws ec2 describe-instances --instance-ids $instanceId --query 'Reservations[0].Instances[0].PublicDnsName' --output text)

# reboot instance, because I was getting "Failed to initialize NVML: Driver/library version mismatch"
# error when running the nvidia-smi command
# see also http://forums.fast.ai/t/no-cuda-capable-device-is-detected/168/13
aws ec2 reboot-instances --instance-ids $instanceId
```

[]